# Julia version of bonding models for PES !!

### Basic Optimization tests

In [1]:
using Optim, LsqFit # opt libs
using DelimitedFiles, MLUtils, BenchmarkTools # utils

In [116]:
# examples of "closures", a technique to fix args!
f(x::Vector, a, b) = (x[1] - a)^2 + (x[2] - b)^2

using Optim
g(x::Vector) = f(x, 3, 4)
optimize(g, [0.,0.])

 * Status: success

 * Candidate solution
    Final objective value:     1.226133e-10

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    50
    f(x) calls:    100


In [117]:
# another closure example in Julia, anonymous function:
optimize(x -> f(x,3,4), [0.,0.]) 

 * Status: success

 * Candidate solution
    Final objective value:     1.226133e-10

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    50
    f(x) calls:    100


In [2]:
# functional form:
function f_ratpot_2(Θ, R, M)
    #=
    ansatz 1 for diatomic potential
    params:
        - Θ := training parameters, vector ()
        - R := distances, vector
    =#
    # unroll coefficients
    a = Θ[1:M]
    b = Θ[M+1:2*M]
    c = Θ[2*M+1:3*M+4]
    d = Θ[3*M+5:4*M+7]
    
    # b_i ≥ 0 for i > 1:
    t = b[2:M]
    bool = t .< 0.
    t[bool] = -t[bool]
    b[2:M] = t
    
    # d_i ≥ 0 for i > 0:
    bool = d .< 0.
    d[bool] = -d[bool]    
    
    # evaluate P:
    P = c[end-1]
    P = P .* ((R .- a[1]).^2 .+ (b[1] .* R))
    for i=2:M
        P .*= (R .- a[i]).^2 .+ (b[i]*R)
    end

    
    # eval Q:
    Q = (R .+ d[end]).*R
    for i=1:M+2
        Q .*= (R .- c[i]).^2 .+ d[i].*R
    end
    
    # eval potential:
    V = c[end] .+ (P ./ Q)
    return V
end

f_RMSE(X, Y) = √(sum((X .- Y) .^ 2)/length(X))

function f_least_squares(f_eval, Y, f_args...)
    Y_pred = f_eval(f_args...)
    res = sum((Y .- Y_pred).^2)
    return res
end

f_least_squares (generic function with 1 method)

In [212]:
M = 2
Θ = collect(LinRange(-1., -2., 21))
Θ = Θ[1:4*M+7] # 4M+7 params
println(Θ, size(Θ))
R = collect(LinRange(0., 5., 6)) .+ 1
println(R)
f_ratpot_2(Θ, R, M)

[-1.0, -1.05, -1.1, -1.15, -1.2000000000000002, -1.25, -1.2999999999999998, -1.35, -1.4, -1.4500000000000002, -1.5, -1.55, -1.6, -1.65, -1.7](15,)
[1.0, 2.0, 3.0, 4.0, 5.0, 6.0]
[-1.1, 1.15]
[1.5, 1.55, 1.6, 1.65, 1.7]


6-element Vector{Float64}:
 -1.4538848497222299
 -1.4504032610414908
 -1.450089540167252
 -1.4500280979713995
 -1.450010836561844
 -1.4500048065726545

#### Optimize ratpot 2 !!

In [3]:
# prepare data:
H_data = readdlm("data/h2/h2_ground_w.txt")
R = H_data[:, 1]; V = H_data[:, 2]
Xs, Ys = shuffleobs((R, V))
train_data, test_data = splitobs((Xs, Ys); at=0.8)
R_train = copy(train_data[1]); V_train = copy(train_data[2]);
R_test = copy(test_data[1]); V_test = copy(test_data[2]);

# tuning parameters:
M = 20
ub = 10; lb = 0
θ = rand(4*M+7) .* (ub-lb) .+ lb # random between [a, b] = [-1, 1]
#println(f_ratpot_2(θ, R_train, M))
f_least_squares(f_ratpot_2, V_train, θ, R_train, M)

4404.864370899371

In [4]:
# optimize:

res = optimize(θ -> f_least_squares(f_ratpot_2, V_train, θ, R_train, M),
                θ, BFGS(),
                Optim.Options(iterations = 2000, show_trace=true); 
                autodiff = :forward
                )

Iter     Function value   Gradient norm 
     0     4.404864e+03     3.062211e+03
 * time: 0.01399993896484375
     1     2.793404e+03     2.440113e+03
 * time: 0.7990000247955322
     2     1.630070e+01     2.054986e+00
 * time: 0.812000036239624
     3     1.629872e+01     8.659113e-03
 * time: 0.8209998607635498
     4     1.623607e+01     5.600086e+00
 * time: 0.8889999389648438
     5     1.623432e+01     1.959398e+00
 * time: 0.9119999408721924
     6     1.621126e+01     8.370857e-01
 * time: 0.9229998588562012
     7     1.620387e+01     1.160661e+00
 * time: 0.9440000057220459
     8     1.619207e+01     1.135099e+00
 * time: 0.9639999866485596
     9     1.618045e+01     3.816732e-01
 * time: 0.9779999256134033
    10     1.613029e+01     1.295419e+01
 * time: 0.9939999580383301
    11     1.584807e+01     5.226360e+01
 * time: 1.0390000343322754
    12     1.584544e+01     5.503163e+01
 * time: 1.0829999446868896
    13     1.573687e+01     5.396900e+01
 * time: 1.0910000801

   120     1.033421e-04     3.278408e-02
 * time: 2.259000062942505
   121     9.759661e-05     2.962836e-02
 * time: 2.2690000534057617
   122     9.418553e-05     1.530917e-02
 * time: 2.2820000648498535
   123     9.204677e-05     3.411398e-03
 * time: 2.2960000038146973
   124     9.096486e-05     2.953289e-03
 * time: 2.302000045776367
   125     9.013070e-05     5.460474e-03
 * time: 2.309000015258789
   126     8.963581e-05     5.160004e-03
 * time: 2.315999984741211
   127     8.943762e-05     1.304418e-03
 * time: 2.3269999027252197
   128     8.911740e-05     3.887631e-03
 * time: 2.3329999446868896
   129     8.708551e-05     2.123379e-02
 * time: 2.3459999561309814
   130     8.373967e-05     2.494811e-02
 * time: 2.3559999465942383
   131     7.825803e-05     3.904916e-02
 * time: 2.365000009536743
   132     7.451538e-05     4.135836e-02
 * time: 2.372999906539917
   133     7.281997e-05     1.607791e-02
 * time: 2.381999969482422
   134     7.124462e-05     2.137468e-02


   240     8.393406e-06     7.189699e-04
 * time: 3.4129998683929443
   241     8.383559e-06     2.693552e-04
 * time: 3.421999931335449
   242     8.373837e-06     8.535068e-04
 * time: 3.430000066757202
   243     8.366244e-06     1.274197e-03
 * time: 3.437999963760376
   244     8.340059e-06     5.983128e-04
 * time: 3.4489998817443848
   245     8.306630e-06     1.754274e-03
 * time: 3.4619998931884766
   246     8.281918e-06     3.587015e-03
 * time: 3.4739999771118164
   247     8.254672e-06     3.657691e-03
 * time: 3.4789998531341553
   248     8.233224e-06     3.390320e-03
 * time: 3.484999895095825
   249     8.171514e-06     2.351461e-03
 * time: 3.493000030517578
   250     8.055552e-06     2.608718e-03
 * time: 3.500999927520752
   251     7.972921e-06     1.852941e-03
 * time: 3.51200008392334
   252     7.846761e-06     6.602139e-04
 * time: 3.5199999809265137
   253     7.793081e-06     3.921411e-03
 * time: 3.5320000648498535
   254     7.688746e-06     2.702694e-03
 

   361     3.878768e-06     1.247204e-03
 * time: 4.6519999504089355
   362     3.813689e-06     2.949838e-03
 * time: 4.662999868392944
   363     3.804321e-06     3.232468e-03
 * time: 4.675999879837036
   364     3.781788e-06     1.996771e-03
 * time: 4.683000087738037
   365     3.769767e-06     2.583854e-03
 * time: 4.691999912261963
   366     3.755177e-06     8.771728e-04
 * time: 4.694999933242798
   367     3.745423e-06     3.559733e-04
 * time: 4.70199990272522
   368     3.730694e-06     1.595028e-03
 * time: 4.710000038146973
   369     3.710211e-06     9.228669e-04
 * time: 4.7170000076293945
   370     3.698558e-06     1.005507e-03
 * time: 4.726999998092651
   371     3.686994e-06     7.179547e-04
 * time: 4.734999895095825
   372     3.675850e-06     6.846601e-04
 * time: 4.747999906539917
   373     3.663203e-06     2.265246e-03
 * time: 4.760999917984009
   374     3.652998e-06     1.846160e-03
 * time: 4.767999887466431
   375     3.630089e-06     1.022246e-03
 * tim

   482     1.444510e-06     1.550290e-03
 * time: 5.703999996185303
   483     1.439209e-06     1.001724e-03
 * time: 5.710999965667725
   484     1.434615e-06     7.841825e-04
 * time: 5.7209999561309814
   485     1.428665e-06     8.014224e-04
 * time: 5.728999853134155
   486     1.426292e-06     2.566635e-04
 * time: 5.740000009536743
   487     1.425648e-06     3.599164e-04
 * time: 5.746999979019165
   488     1.425435e-06     1.448209e-04
 * time: 5.756999969482422
   489     1.424800e-06     1.482361e-04
 * time: 5.766999959945679
   490     1.423052e-06     6.162086e-04
 * time: 5.772000074386597
   491     1.417523e-06     6.600706e-04
 * time: 5.7809998989105225
   492     1.391232e-06     1.721418e-03
 * time: 5.792999982833862
   493     1.382971e-06     1.405119e-03
 * time: 5.796000003814697
   494     1.367367e-06     5.009336e-04
 * time: 5.802000045776367
   495     1.351493e-06     9.023372e-04
 * time: 5.812000036239624
   496     1.329956e-06     1.659382e-03
 * ti

...

  1567     2.784578e-08     2.654445e-05
 * time: 14.477999925613403
  1568     2.782327e-08     7.257762e-05
 * time: 14.490999937057495
  1569     2.777407e-08     7.983030e-05
 * time: 14.49399995803833
  1570     2.776381e-08     7.072581e-05
 * time: 14.497999906539917
  1571     2.772000e-08     1.046459e-04
 * time: 14.520999908447266
  1572     2.767471e-08     1.187270e-04
 * time: 14.526999950408936
  1573     2.763844e-08     5.002302e-05
 * time: 14.52999997138977
  1574     2.760715e-08     8.637362e-05
 * time: 14.53600001335144
  1575     2.755464e-08     2.045461e-04
 * time: 14.54099988937378
  1576     2.740473e-08     2.875749e-04
 * time: 14.549000024795532
  1577     2.730739e-08     3.299978e-04
 * time: 14.552000045776367
  1578     2.713868e-08     1.438670e-04
 * time: 14.558000087738037
  1579     2.687864e-08     1.127424e-04
 * time: 14.562999963760376
  1580     2.659307e-08     2.083656e-04
 * time: 14.57800006866455
  1581     2.649832e-08     2.443378e-0

  1687     1.981331e-08     8.469641e-05
 * time: 15.389999866485596
  1688     1.980466e-08     1.405067e-05
 * time: 15.398000001907349
  1689     1.979199e-08     5.296152e-05
 * time: 15.40499997138977
  1690     1.978145e-08     6.558709e-05
 * time: 15.41599988937378
  1691     1.976345e-08     6.376730e-05
 * time: 15.420000076293945
  1692     1.975420e-08     3.834053e-05
 * time: 15.430000066757202
  1693     1.973670e-08     7.596640e-05
 * time: 15.437000036239624
  1694     1.971636e-08     7.929444e-06
 * time: 15.44700002670288
  1695     1.970755e-08     2.523706e-05
 * time: 15.450000047683716
  1696     1.969432e-08     6.375197e-05
 * time: 15.455999851226807
  1697     1.968753e-08     2.293858e-05
 * time: 15.460000038146973
  1698     1.968309e-08     6.615063e-05
 * time: 15.466000080108643
  1699     1.967459e-08     6.693854e-05
 * time: 15.470999956130981
  1700     1.966110e-08     2.352684e-05
 * time: 15.480000019073486
  1701     1.965461e-08     5.441345e

  1807     1.471043e-08     1.024274e-04
 * time: 16.277999877929688
  1808     1.465740e-08     1.799933e-04
 * time: 16.287999868392944
  1809     1.462866e-08     5.048260e-05
 * time: 16.2979998588562
  1810     1.459440e-08     1.190599e-04
 * time: 16.305000066757202
  1811     1.458071e-08     1.277944e-04
 * time: 16.308000087738037
  1812     1.455399e-08     1.497116e-04
 * time: 16.312000036239624
  1813     1.451666e-08     1.368081e-04
 * time: 16.32200002670288
  1814     1.448424e-08     1.456920e-04
 * time: 16.33299994468689
  1815     1.446093e-08     5.958031e-05
 * time: 16.335999965667725
  1816     1.443532e-08     1.817367e-05
 * time: 16.33899998664856
  1817     1.441787e-08     1.828646e-05
 * time: 16.346999883651733
  1818     1.439882e-08     5.534342e-05
 * time: 16.35599994659424
  1819     1.436990e-08     1.373292e-04
 * time: 16.36299991607666
  1820     1.434943e-08     1.606989e-04
 * time: 16.369999885559082
  1821     1.433930e-08     2.305833e-04


  1927     1.199080e-08     9.394632e-05
 * time: 17.149999856948853
  1928     1.197570e-08     1.713417e-05
 * time: 17.152999877929688
  1929     1.195674e-08     5.532750e-05
 * time: 17.16000008583069
  1930     1.195024e-08     7.004180e-05
 * time: 17.162999868392944
  1931     1.193227e-08     9.310323e-05
 * time: 17.168999910354614
  1932     1.190855e-08     9.979035e-05
 * time: 17.18400001525879
  1933     1.188438e-08     1.145837e-04
 * time: 17.194000005722046
  1934     1.185812e-08     2.248771e-05
 * time: 17.204999923706055
  1935     1.184291e-08     3.051145e-05
 * time: 17.2189998626709
  1936     1.181906e-08     1.032138e-04
 * time: 17.22499990463257
  1937     1.180969e-08     4.028805e-05
 * time: 17.227999925613403
  1938     1.179983e-08     1.040133e-04
 * time: 17.240000009536743
  1939     1.178199e-08     9.041857e-05
 * time: 17.250999927520752
  1940     1.173953e-08     2.705260e-05
 * time: 17.256999969482422
  1941     1.173241e-08     2.045151e-0

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     9.611939e-09

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 5.21e-04 ≰ 0.0e+00
    |x - x'|/|x'|          = 3.83e-05 ≰ 0.0e+00
    |f(x) - f(x')|         = 3.00e-11 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 3.12e-03 ≰ 0.0e+00
    |g(x)|                 = 7.12e-05 ≰ 1.0e-08

 * Work counters
    Seconds run:   18  (vs limit Inf)
    Iterations:    2000
    f(x) calls:    5173
    ∇f(x) calls:   5173


In [5]:
# check RMSE:
V_pred = f_ratpot_2(res.minimizer, R_train, M)
println(f_RMSE(V_train, V_pred))
V_pred = f_ratpot_2(res.minimizer, R_test, M)
println(f_RMSE(V_test, V_pred))
for i=1:length(V_test)
    println(V_pred[i]," ",V_test[i])
end

4.234704519894572e-6
2.9276919358380464e-6
-0.0008473614097407878 -0.0008457269999999628
-0.07954291804404812 -0.07954594900000012
-0.1519622732506834 -0.15195909900000015
-0.11653569803675493 -0.1165328080000001
-0.03391427554983345 -0.03391220100000014
-0.005351304696011028 -0.005348754000000122
-0.16747835744279463 -0.16748270900000017
-0.17098887938290577 -0.17099233300000005
-5.478405504333317e-5 -5.306299999996433e-5
-0.031062824242911735 -0.031061373000000003
-0.010857211378577925 -0.010857708000000077
-0.026627807954423488 -0.026627746000000174
-0.15506383270077323 -0.15506615300000015
-0.07577424840002328 -0.07577727099999998
-8.232465193436234e-6 -6.208000000063052e-6
-0.0006030500384439641 -0.0006003549999999969
-0.0010596603561299767 -0.001059486000000165
-0.01959761557770706 -0.019599871000000046
-9.81964487805631e-5 -9.699600000012687e-5
-0.021836953193801134 -0.02183858500000002
-0.15661983418865094 -0.15662260100000003
-0.03696914403296936 -0.036966681
-0.16865910473891

In [210]:
function fmin(x,a,b,c)
   return sum(x.^2) + a*b + c
end

a0 = 3.
b0 = 2.
c0 = 4.
res = optimize(x -> fmin(x,a0,b0,c0), [100., 100.], LBFGS(); autodiff = :forward)
println(res)
println(res.minimizer, res.minimum)
#println(f(res.minimizer, a0, b0))

 * Status: success

 * Candidate solution
    Final objective value:     1.000000e+01

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 1.00e+02 ≰ 0.0e+00
    |x - x'|/|x'|          = Inf ≰ 0.0e+00
    |f(x) - f(x')|         = 2.00e+04 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 2.00e+03 ≰ 0.0e+00
    |g(x)|                 = 0.00e+00 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1
    f(x) calls:    3
    ∇f(x) calls:   3

[0.0, 0.0]10.0


### 8.1 Bonding features

In [75]:
function t_R_fun(R, R_up, R_low, e)
    R2 = R.^2
    return ((R2 .- R_low^2)./(R_up^2 .- R2)).^e
end

#function s_bond_strength

t_R_fun (generic function with 1 method)

In [76]:
R = [2.5, 3.1, 5, 1.1]
@benchmark t_R_fun(R, 6, 1, 2)

BenchmarkTools.Trial: 10000 samples with 982 evaluations.
 Range (min … max):  59.369 ns …  1.314 μs  ┊ GC (min … max): 0.00% … 92.58%
 Time  (median):     61.507 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   67.030 ns ± 56.162 ns  ┊ GC (mean ± σ):  5.19% ±  5.87%

  ▄██▄▅▄▃▂▁▁                                                  ▂
  ███████████▇▇▇▆▆▆▅▄▅▆▄▄▄▆▄▃▅▄▁▃▁▄▁▁▃▄▃▃▁▁▁▁▁▃▁▁▃▁▁▃▅▆▇▇██▇▇ █
  59.4 ns      Histogram: log(frequency) by time       119 ns <

 Memory estimate: 192 bytes, allocs estimate: 2.